In [ ]:
import pprint as pp
import pandas as pd
import os
import folium
import folium.plugins

# Set the working directory
os.chdir(r'C:\Users\mikke\Documents\GitHub\graphs')

filepath = 'C:/Users/mikke/Desktop/trafficdata/US_Accidents_March23.csv'

# Load the dataset
accident_data = pd.read_csv(filepath)

# Group by state and get their sizes
# state_sizes = accident_data.groupby('State').size()
# state_sizes.to_csv('analysis_data/state_sizes.csv')




In [ ]:
filepath = 'C:/Users/mikke/Desktop/trafficdata/US_Accidents_March23.csv'

# Load the dataset
accident_data = pd.read_csv(filepath)

county_sizes = accident_data.groupby('County').size()
county_sizes.to_csv('analysis_data/county_sizes.csv')


In [ ]:
# Load the staterenamings.csv file
staterenamings = pd.read_csv('staterenamings.csv')

state_sizes = pd.read_csv('analysis_data/state_sizes.csv', header=None, names=['State', 'count'])

# Merge the state_sizes with the staterenamings
state_sizes = state_sizes.reset_index()
state_sizes = state_sizes.rename(columns={0: 'count'})
state_sizes = state_sizes.merge(staterenamings, left_on='State', right_on='Abbreviation')




In [ ]:
import folium
import pandas as pd

# Create a map centered on the USA
map_usa = folium.Map(location=[37.0902, -95.7129], zoom_start=4)

# Add choropleth layer to the map
folium.Choropleth(
    geo_data='us-states.json',  # GeoJSON file containing state boundaries
    name='choropleth',
    data=state_sizes,
    columns=['State_y', 'count'],
    key_on='feature.properties.name',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='State Traffic Accidents'
).add_to(map_usa)

# Display the map
map_usa


In [ ]:
import folium
import pandas as pd

# Load the county_sizes.csv file
county_sizes = pd.read_csv('analysis_data\county_sizes.csv')

county_sizes.rename(columns={'Unnamed: 0': 'count'}, inplace=True)

county_sizes.columns = ['County', 'count']

# Create a map centered on the USA
map_usa = folium.Map(location=[37.0902, -95.7129], zoom_start=4)

# Add choropleth layer to the map
folium.Choropleth(
    geo_data='counties.geojson',  # GeoJSON file containing county boundaries
    name='choropleth',
    data=county_sizes,
    columns=['County', 'count'],
    key_on='feature.properties.NAME',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='County Traffic Accidents'
).add_to(map_usa)

# Display the map
map_usa

In [ ]:
import geopandas as gpd





# LA_data = accident_data[accident_data['City'] == 'Los Angeles']

# LA_data.to_csv('analysis_data/LA_data.csv')



In [30]:
import pandas as pd
import folium

LA_data = pd.read_csv('analysis_data/LA_data.csv')

# # Create a map centered on Los Angeles
# map_LA = folium.Map(location=[34.0522, -118.2437], zoom_start=10)
# for index, row in LA_data.iterrows():
#     folium.Marker(
#         location=[row['Start_Lat'], row['Start_Lng']],
#         popup=row['Description'],
#         icon=folium.Icon(color='red', icon='info-sign')
#     ).add_to(map_LA)

# # Save the map as an image file
# map_LA.save('map_LA.png')


gdf = gpd.read_file(r"data\network['Los Angeles , California, USA'].gpkg", layer='nodes')

gdf.columns

print(LA_data.columns)

null_proportion = LA_data.isnull().mean() * 100
print(null_proportion)

print(LA_data['Station'].value_counts())

Index(['Unnamed: 0', 'ID', 'Source', 'Severity', 'Start_Time', 'End_Time',
       'Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng', 'Distance(mi)',
       'Description', 'Street', 'City', 'County', 'State', 'Zipcode',
       'Country', 'Timezone', 'Airport_Code', 'Weather_Timestamp',
       'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)',
       'Visibility(mi)', 'Wind_Direction', 'Wind_Speed(mph)',
       'Precipitation(in)', 'Weather_Condition', 'Amenity', 'Bump', 'Crossing',
       'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station',
       'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop',
       'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight'],
      dtype='object')
Unnamed: 0                0.000000
ID                        0.000000
Source                    0.000000
Severity                  0.000000
Start_Time                0.000000
End_Time                  0.000000
Start_Lat                 0

In [1]:
import geopandas as gpd
from shapely.geometry import Point
import numpy as np
from shapely.geometry import Point
import geopandas as gpd
import pandas as pd

LA_data = pd.read_csv('analysis_data/LA_data.csv')
gdf = gpd.read_file(r"data\network['Los Angeles , California, USA'].gpkg", layer='nodes')

# Create the geometry column using the Start_Lat and Start_Lng columns
geometry = [Point(xy) for xy in zip(LA_data['Start_Lng'], LA_data['Start_Lat'])]



# Create the GeoDataFrame
LA_gdf = gpd.GeoDataFrame(LA_data, geometry=geometry)


print(gdf.crs)
LA_gdf.set_crs(epsg=4326, inplace=True)
LA_gdf = LA_gdf.to_crs(epsg=6423)
gdf = gdf.to_crs(epsg=6423)


LA_gdf['geometry'].head()



EPSG:4326


0    POINT (1973322.115 564256.250)
1    POINT (1977900.475 565603.354)
2    POINT (1971458.800 559637.732)
3    POINT (1974649.461 558543.528)
4    POINT (1974168.217 549680.495)
Name: geometry, dtype: geometry

In [3]:
import geopandas as gpd
from shapely.geometry import Point

def add_accident_counts(gdf, LA_gdf, range_):
    gdf = gdf.copy()
    LA_gdf = LA_gdf.copy()
    gdf['geometry'] = gdf.geometry.buffer(range_)
    joined = gpd.sjoin(LA_gdf, gdf, how='inner', op='intersects')
    
    def nearest_intersection(accident):
        intersections = joined[joined['ID'] == accident].index_right
        if not intersections.empty:
            nearest = min(intersections, key=lambda intersection: LA_gdf[LA_gdf['ID'] == accident].geometry.distance(gdf.loc[intersection].geometry).min())
            return nearest
        else:
            return None

    LA_gdf['nearest_intersection'] = LA_gdf['ID'].apply(nearest_intersection)
    accident_counts = LA_gdf['nearest_intersection'].value_counts()
    gdf['accident_count'] = gdf.index.map(accident_counts.to_dict())
    gdf['accident_count'] = gdf['accident_count'].fillna(0)
    gdf.to_file(f'analysis_data/intersections_with_accidents_range{range_}.gpkg', driver='GPKG')

range_list = [2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 30, 40, 50]

for i in range_list:
    add_accident_counts(gdf, LA_gdf, i)
    print(f'Range {i} meters done')






C:\Users\mikke\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py:3548: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


Range 2 meters done


C:\Users\mikke\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py:3548: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


Range 3 meters done


C:\Users\mikke\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py:3548: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


...

Range 30 meters done


C:\Users\mikke\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py:3548: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


Range 40 meters done


C:\Users\mikke\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py:3548: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


Range 50 meters done


In [23]:
import numpy as np

gdf = gpd.read_file('analysis_data/intersections_with_accidents_range5.gpkg')

accidents_above_zero = gdf[gdf['accident_count'] > 0].sort_values(by='accident_count', ascending=False)
print(accidents_above_zero['accident_count'])


35314    203.0
446      124.0
36751    111.0
863      106.0
35708     92.0
         ...  
19747      1.0
19801      1.0
19802      1.0
19859      1.0
49408      1.0
Name: accident_count, Length: 3381, dtype: float64


In [31]:
# Create a map centered on Los Angeles
map_LA = folium.Map(location=[34.0522, -118.2437], zoom_start=10)

# Iterate over each row in the accidents_above_zero GeoDataFrame
for index, row in accidents_above_zero.iterrows():
    # Get the coordinates and accident count
    lat = row['y']
    lon = row['x']
    accident_count = row['accident_count']
    
    # Create a CircleMarker with a radius based on the accident count
    radius = 5  # Adjust the multiplier as needed
    color = 'blue' if accident_count > 50 else 'red' if accident_count > 10 else 'orange' if accident_count > 5 else 'yellow' if accident_count >= 0 else 'green'
    circle_marker = folium.CircleMarker(location=[lat, lon], radius=radius, color=color, fill=True, fill_color=color)
    
    # Add the CircleMarker to the map
    circle_marker.add_to(map_LA)

# Display the map
map_LA


In [22]:
gdf.shape

(49456, 8)